<a href="https://colab.research.google.com/github/nachomendoza97/Data-Industrial/blob/main/APP_OPT_Sheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
##Codigo de vinculacion de aplicaciones con plantillas de Google Sheets "Gestion de datos Industrial"
# --- 1. CONFIGURACIÓN Y AUTENTICACIÓN ---
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# --- 2. CONEXIÓN AL ARCHIVO (Usando tu ID real) ---
ID_HOJA = "1aPsyywIl1GXf7R_GONxx8CCCqVS4gEenTWHtylUVUA8"
wb = gc.open_by_key(ID_HOJA)
sheet_inp = wb.worksheet("INP_OPT-Scheduling")

def app_scheduling_opt():
    # A. Lectura de Matriz (Rango A1:K11)
    datos_matriz = sheet_inp.get('A1:K11')
    matriz_df = pd.DataFrame(datos_matriz[1:], columns=datos_matriz[0]).set_index('Lote')

    # Limpieza de espacios en índices y columnas para evitar errores de búsqueda
    matriz_df.columns = matriz_df.columns.str.strip()
    matriz_df.index = matriz_df.index.str.strip().str.replace('*', '') # Remove asterisks from index
    matriz_df = matriz_df.replace('N/A', 0).astype(float)

    # Debugging: Print index and columns of matriz_df
    print("Índice de matriz_df:", matriz_df.index.tolist())
    print("Columnas de matriz_df:", matriz_df.columns.tolist())

    # B. Lectura de Lotes (Rango A14:G25)
    datos_lotes = sheet_inp.get('A14:G25')
    df_lotes = pd.DataFrame(datos_lotes[1:], columns=datos_lotes[0])

    # IMPORTANTE: Limpiamos los nombres de las columnas para evitar el KeyError
    df_lotes.columns = df_lotes.columns.str.strip()

    # C. Conversión a numérico (Aseguramos que coincidan con los nombres en Sheets)
    # Si sigue dando error aquí, verifica que en Sheets no haya un espacio extra al final
    col_proc = 'Tiempo de Procesamiento (min)'
    col_setup = 'Tiempo de Set Up (min)' # Removed '#' from the column name

    df_lotes[col_proc] = pd.to_numeric(df_lotes[col_proc])
    df_lotes[col_setup] = pd.to_numeric(df_lotes[col_setup])

    # D. Lógica de Programación (SPT + Short Tail)
    df_ordenado = df_lotes.sort_values(
        by=[col_proc, col_setup],
        ascending=[True, True]
    ).reset_index(drop=True)

    # Debugging: Print columns of df_ordenado to check for 'Lote ID'
    print("Columnas en df_ordenado:", df_ordenado.columns.tolist())
    print("Lote ID únicos en df_ordenado:", df_ordenado['Lote ID'].unique().tolist())

    cronograma = []
    reloj = 0
    lote_prev = None

    for _, fila in df_ordenado.iterrows():
        lote_id = fila['Lote ID']
        # Buscamos el tiempo de cambio en la matriz
        co = 0 if lote_prev is None else matriz_df.loc[lote_prev, lote_id]

        inicio = reloj + co
        duracion = fila[col_proc] + fila[col_setup]
        fin = inicio + duracion

        cronograma.append({
            'Lote ID': lote_id,
            'ChangeOver_min': co,
            'Inicio_min': inicio,
            'Fin_min': fin
        })
        reloj = fin
        lote_prev = lote_id

    # E. Exportar Resultados a "Resultado_Gantt"
    df_res = pd.DataFrame(cronograma)
    res_sheet = wb.worksheet("Resultado_Gantt")
    res_sheet.clear()
    res_sheet.update([df_res.columns.values.tolist()] + df_res.values.tolist())

    print("✅ Programación de producción exportada exitosamente a 'Resultado_Gantt'.")

# --- 3. EJECUCIÓN ---
app_scheduling_opt()

Índice de matriz_df: ['L-01', 'L-02', 'L-03', 'L-04', 'L-05', 'L-06', 'L-07', 'L-08', 'L-09', 'L-10']
Columnas de matriz_df: ['L-01', 'L-02', 'L-03', 'L-04', 'L-05', 'L-06', 'L-07', 'L-08', 'L-09', 'L-10']
Columnas en df_ordenado: ['Lote ID', 'Producto/SKU', 'Tiempo de Procesamiento (min)', 'Tiempo de Set Up (min)', 'Rendimiento por Hora (Unidades)', 'Máquina/Línea']
Lote ID únicos en df_ordenado: ['L-09', 'L-04', 'L-06', 'L-02', 'L-07', 'L-10', 'L-01', 'L-03', 'L-05', 'L-08']
✅ Programación de producción exportada exitosamente a 'Resultado_Gantt'.


In [ ]:
datos_lotes = sheet_inp.get('A14:G25')
df_lotes_temp = pd.DataFrame(datos_lotes[1:], columns=datos_lotes[0])
print('Columnas encontradas en la hoja de cálculo:')
print(df_lotes_temp.columns.tolist())

Columnas encontradas en la hoja de cálculo:
['Lote ID', 'Producto/SKU', 'Tiempo de Procesamiento (min)', 'Tiempo de Set Up (min)', 'Rendimiento por Hora (Unidades)', 'Máquina/Línea']


In [10]:
import gspread
from google.colab import auth
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Reemplaza 'TU_ID_DE_HOJA_DE_CALCULO' con el ID real de tu hoja de cálculo
spreadsheet_id = 'TU_ID_DE_HOJA_DE_CALCULO'
wb = gc.open_by_id(spreadsheet_id)

# Continúa con el resto de tu código, por ejemplo:
sheet_inp = wb.worksheet("INP_OPT-Scheduling")
print("Conexión exitosa a la hoja de cálculo por ID.")

AttributeError: 'Client' object has no attribute 'open_by_id'

Recuerda que deberás reemplazar `TU_ID_DE_HOJA_DE_CALCULO` con el ID real de tu hoja de cálculo.